In [ ]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import numpy as np
from data_loader import load_mnist
import matplotlib.pyplot as plt

In [ ]:
print("Loading data...")
X_train, y_train, X_test, y_test = load_mnist()
print("MNIST Data Loaded.")

In [ ]:
# Check Shapes and Types
print(f"Training Data Shape: {X_train.shape}")  # Should be (60000, 784)
print(f"Test Data Shape: {X_test.shape}")       # Should be (10000, 784)
print(f"Data Type: {X_train.dtype}")            # Should be float32
print(f"Min Value: {X_train.min()}")            # Should be 0.0
print(f"Max Value: {X_train.max()}")            # Should be 1.0

# We need to reshape the flat (784,) vectors back to (28, 28) images to plot
print("\nVisualising 5 Random Digits:")

indices = np.random.randint(0, len(X_train), 5)
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i, ax in zip(indices, axes):
    image = X_train[i].reshape(28, 28)
    label = y_train[i]
    
    ax.imshow(image, cmap='gray')
    ax.set_title(f"{label}")
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from vae import VAE
import time

In [ ]:
BATCH_SIZE = 128
EPOCHS = 15
LEARNING_RATE = 0.001
K_LATENT = 32
HIDDEN_LAYER_DIM = 512
ALPHA_HP = 1.0

vae = VAE(n=784, hidden_layer_dim=HIDDEN_LAYER_DIM, k_latent=K_LATENT, learning_rate=LEARNING_RATE)

print(f"Starting training for {EPOCHS} epochs...")
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_loss = 0
    epoch_recon = 0
    epoch_kl = 0

    indices = np.arange(X_train.shape[0])
    np.random.shuffle(indices)
    X_train_shuffled = X_train[indices]

    # Mini-batch Gradient Descent
    for i in range(0, X_train.shape[0], BATCH_SIZE):
        # Get batch
        X_batch = X_train_shuffled[i : i + BATCH_SIZE]
        
        # Forward Pass
        X_hat, mu, sigma = vae.forward_pass(X_batch)
        
        # Backward Pass
        vae.backward_pass(alpha_hyper_param=ALPHA_HP)
        
        # Update Weights
        # vae.GD_update_step()
        vae.adam_step()

        # Track Loss (MSE + KL) for monitoring
        recon_loss = np.mean(np.sum((X_batch - X_hat)**2, axis=1))
        kl_loss = np.mean(-0.5 * np.sum(1 + np.log(sigma**2) - mu**2 - sigma**2, axis=1))
        total_loss = recon_loss + (ALPHA_HP * kl_loss)
        
        epoch_loss += total_loss
        epoch_recon += recon_loss
        epoch_kl += kl_loss

    # Print average loss for the epoch
    n_batches = X_train.shape[0] / BATCH_SIZE
    avg_loss = epoch_loss / n_batches
    avg_recon = epoch_recon / n_batches
    avg_kl = epoch_kl / n_batches
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Total Loss: {avg_loss:.2f} | Recon Loss: {avg_recon:.2f} | KL Loss: {avg_kl:.2f}")

print(f"Training finished in {time.time() - start_time:.2f} seconds.")

In [ ]:
vae.model_infer(num_samples=5)

In [ ]:
vae.save_model(filename="vae_mnist")